In [ ]:
%matplotlib inline
import os
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import numpy as np
import pandas as pd
import json
from tqdm import tqdm, tqdm_notebook
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout, Flatten, Dense
from tensorflow.keras.layers import GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import optimizers

## Introduction
There are billions of humans on this earth, and each of us is made up of trillions of cells. Just like every individual is unique, even genetically identical twins, scientists observe  differences between the genetically identical cells in our bodies.  

> この地球には何十億もの人間がいて、私たち一人一人は何兆もの細胞で構成されています。  
すべての個体がユニークであるように、遺伝的に同一の双子でさえ、  
科学者は私たちの体の遺伝的に同一の細胞間の違いを観察します。

Differences in the location of proteins can give rise to such cellular heterogeneity. Proteins play essential roles in virtually all cellular processes. Often, many different proteins come together at a specific location to perform a task, and the exact outcome of this task depends on which proteins are present. As you can imagine, different subcellular distributions of one protein can give rise to great functional heterogeneity between cells. Finding such differences, and figuring out how and why they occur, is important for understanding how cells function, how diseases develop, and ultimately how to develop better treatments for those diseases.

> タンパク質の位置の違いは、そのような細胞の不均一性を引き起こす可能性があります。  
タンパク質は、事実上すべての細胞プロセスで重要な役割を果たします。  
多くの場合、多くの異なるタンパク質が特定の場所に集まってタスクを実行します。  
このタスクの正確な結果は、どのタンパク質が存在するかによって異なります。  
ご想像のとおり、1つのタンパク質の細胞内分布が異なると、細胞間に大きな機能的不均一性が生じる可能性があります。  
そのような違いを見つけ、それらがどのようにそしてなぜ起こるのかを理解することは、細胞がどのように機能するか、  
病気がどのように進行するか、そして最終的にはそれらの病気のより良い治療法を開発する方法を理解するために重要です。

To see more, start with less. That may seem counterintuitive, but the study of a single cell enables the discovery of mechanisms too difficult to see with multi-cell research. The importance of studying single cells is reflected in the ongoing revolution in biology centered around technologies for single cell analysis. Microscopy offers an opportunity to study differences in protein localizations within a population of cells. Current machine learning models for classifying protein localization patterns in microscope images gives a summary of the entire population of cells. However, the single-cell revolution in biology demands models that can precisely classify patterns in each individual cell in the image.

> もっと見るには、少ないものから始めてください。  
それは直感に反するように思えるかもしれませんが、単一細胞の研究は、複数細胞の研究では見るのが難しすぎるメカニズムの発見を可能にします。  
単一細胞を研究することの重要性は、単一細胞分析の技術を中心とした生物学の進行中の革命に反映されています。  
顕微鏡検査は、細胞集団内のタンパク質局在の違いを研究する機会を提供します。  
顕微鏡画像のタンパク質局在パターンを分類するための現在の機械学習モデルは、細胞の全集団の要約を提供します。   
ただし、生物学における単一細胞の革命には、画像内の個々の細胞のパターンを正確に分類できるモデルが必要です。

## Submission

Submissions are evaluated by computing macro F1, with the mean taken over the 19 segmentable classes of the challenge. It is otherwise essentially identical to the OpenImages Instance Segmentation Challenge evaluation. The OpenImages version of the metric is described in detail here. See also this tutorial on running the evaluation in Python, with the only difference being the use of F1 rather than average precision.

Segmentation is calculated using IoU with a threshold of 0.6.

> 提出物は、マクロF1を計算することによって評価され、  
平均はチャレンジの19のセグメント化可能なクラスにわたって取得されます。  
それ以外は、OpenImagesインスタンスセグメンテーションチャレンジの評価と本質的に同じです。  
メトリックのOpenImagesバージョンについては、ここで詳しく説明します。   
Pythonでの評価の実行に関するこのチュートリアルも参照してください。  
唯一の違いは、平均精度ではなくF1を使用することです。  
セグメンテーションは、0.6のしきい値でIoUを使用して計算されます。

## Submission File
For each image in the test set, you must predict a list of instance segmentation masks and their associated detection score (Confidence). The submission csv file uses the following format:
> テストセット内の各画像について、  
インスタンスセグメンテーションマスクのリストとそれに関連する検出スコア（信頼度）を予測する必要があります。  送信csvファイルは次の形式を使用します。

In [ ]:
train_df = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
train_df

## What am I predicting?

You are predicting protein organelle localization labels for each cell in the image. Border cells are included when there is enough information to decide on the labels.

> 画像内の各細胞のタンパク質オルガネラ局在化ラベルを予測しています。ラベルを決定するのに十分な情報がある場合、境界セルが含まれます。

There are in total 19 different labels present in the dataset (18 labels for specific locations, and label 18 for negative and unspecific signal). The dataset is acquired in a highly standardized way using one imaging modality (confocal microscopy). However, the dataset comprises 17 different cell types of highly different morphology, which affect the protein patterns of the different organelles. All image samples are represented by four filters (stored as individual files), the protein of interest (green) plus three cellular landmarks: nucleus (blue), microtubules (red), endoplasmic reticulum (yellow). The green filter should hence be used to predict the label, and the other filters are used as references. The labels are represented as integers that map to the following:

> データセットには合計19の異なるラベルが存在します（特定の場所に18のラベル、負の非特定の信号に18のラベル）。データセットは、1つのイメージングモダリティ（共焦点顕微鏡）を使用して、高度に標準化された方法で取得されます。ただし、データセットは、さまざまな細胞小器官のタンパク質パターンに影響を与える、非常に異なる形態の17の異なる細胞タイプで構成されています。  
  
> すべての画像サンプルは、4つのフィルター（個別のファイルとして保存）、  
  
>目的のタンパク質（緑）、  
3つの細胞ランドマーク（核（青）、 微小管（赤）、小胞体（黄色））  
で表されます。  
  
>したがって、緑色のフィルターを使用してラベルを予測し、他のフィルターを参照として使用する必要があります。  
ラベルは、以下にマップされる整数として表されます。


0. Nucleoplasm : 核質
1. Nuclear membrane : 核膜
2. Nucleoli : 核小体
3. Nucleoli fibrillar center : 核フィブリル
4. Nuclear speckles : 核スペクル
5. Nuclear bodies : 核内構造体
6. Endoplasmic reticulum : 小胞体
7. Golgi apparatus : ゴルジ体
8. Intermediate filaments : 中間径フィラメント
9. Actin filaments : マイクロフィラメント
10. Microtubules : 微小管
11. Mitotic spindle : 紡錘体
12. Centrosome : 中心体
13. Plasma membrane : 細胞膜
14. Mitochondria : ミトコンドリア
15. Aggresome : アグリソーム
16. Cytosol : 細胞質基質
17. Vesicles and punctate cytosolic patterns : 小胞 and 点状の細胞質ゾルパターン
18. Negative : 陰性

## What files do I need?
On the data page below, you will find a set of full size original images (a mix of 1728x1728, 2048x2048 and 3072x3072 PNG files) in train.zip and test.zip. (Please note that since this is a code competition, part of test data will be hidden)

> 以下のデータページでは、train.zipとtest.zipにフルサイズの元の画像のセット（1728x1728、2048x2048、3072x3072のPNGファイルの組み合わせ）があります。 （これはコードの競合であるため、テストデータの一部が非表示になることに注意してください）

You will also need the image level labels from train.csv and the filenames for the test set from sample_submission.csv. As many Kagglers made use of all public images in HPA for previous classification challenge, we made the public HPA images available to download as instructed in this notebook. Note also that there are TFRecords available if competitors would like to use TPUs.

> また、train.csvの画像レベルのラベルとsample_submission.csvのテストセットのファイル名も必要になります。多くのKagglerが以前の分類チャレンジでHPAのすべての公開画像を利用したため、このノートブックの指示に従って、公開HPA画像をダウンロードできるようにしました。競合他社がTPUを使用したい場合は、TFRecordを利用できることにも注意してください。

## What should I expect the data format to be?

The training image-level labels are provided for each sample in train.csv.  
The bulk of the data for images - train.zip. Each sample consists of four files. Each file represents a different filter on the subcellular protein patterns represented by the sample. The format should be [filename]_[filter color].png for the PNG files.   
Colors are red for   
microtubule channels,   
blue for nuclei channels,   
yellow for Endoplasmic Reticulum (ER) channels,   
and green for protein of interest.

> トレーニング画像レベルのラベルは、train.csvのサンプルごとに提供されます。  
画像のデータの大部分-train.zip。各サンプルは4つのファイルで構成されています。  
各ファイルは、サンプルによって表される細胞内タンパク質パターンの異なるフィルターを表します。  
PNGファイルの形式は[ファイル名] _ [フィルターカラー] .pngである必要があります。  
  
>色は、  
微小管チャネルの場合は赤、  
核チャネルの場合は青、  
小胞体（ER）チャネルの場合は黄色、  
目的のタンパク質の場合は緑です。

## What is meant by weak image-level labels?

The labels you will get for training are image level labels while the task is to predict cell level labels. That is to say, each training image contains a number of cells that have collectively been labeled as described above and the prediction task is to look at images of the same type and predict the labels of each individual cell within those images.

> トレーニング用に取得するラベルは画像レベルのラベルですが、タスクはセルレベルのラベルを予測することです。  つまり、各トレーニング画像には、上記のようにまとめてラベル付けされた多数のセルが含まれ、  
予測タスクは、同じタイプの画像を調べて、それらの画像内の個々のセルのラベルを予測することです。

As the training labels are a collective label for all the cells in an image, it means that each labeled pattern can be seen in the image but not necessarily that each cell within the image expresses the pattern. This imprecise labeling is what we refer to as weak.

> トレーニングラベルは画像内のすべてのセルの集合ラベルであるため、  
ラベル付けされた各パターンが画像内に表示されますが、画像内の各セルがパターンを表すとは限りません。  
この不正確なラベル付けは、私たちが弱いと呼ぶものです。

During the challenge you will both need to segment the cells in the images and predict the labels of those segmented cells.

> チャレンジ中は、画像内のセルをセグメント化し、  
それらのセグメント化されたセルのラベルを予測する必要があります。

## Files

|   |   |
|---|---|
|train | training images (in .tif)|
|test  | test images (in .png) the task of the competition is to segment and label the images in this folder|
|train.csv | filenames and image level labels for the training set|  
|sample_submission.csv | filenames for the test set, and a guide to constructing a working submission.|

|   |   |
|---|---|
|train | トレーニング画像（.tif内）|
|test  | テスト画像（.png）コンテストのタスクは、このフォルダー内の画像をセグメント化してラベルを付けることです。|
|train.csv | トレーニングセットのファイル名と画像レベルのラベル|  
|sample_submission.csv | テストセットのファイル名、および実用的な提出物を作成するためのガイド。|

In [ ]:
fig = plt.figure(figsize=(20,24))
imagepath=[]

for file in glob.glob('../input/hpa-single-cell-image-classification/train/*.png')[:50]:
    imagepath.append(file)
    
imagedata=[]

for i in range(36):
    plt.subplot(6, 6, i+1)
    img = cv2.imread(imagepath[i])
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.tick_params(labelbottom=False,
                    labelleft=False,
                    labelright=False,
                    labeltop=False)

In [ ]:
fig = plt.figure(figsize=(20,24))
imagepath=[]

for file in glob.glob('../input/hpa-single-cell-image-classification/test/*.png')[:50]:
    imagepath.append(file)
    
imagedata=[]

for i in range(36):
    plt.subplot(6, 6, i+1)
    img = cv2.imread(imagepath[i])
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.tick_params(labelbottom=False,
                    labelleft=False,
                    labelright=False,
                    labeltop=False)

In [ ]:
train_df = pd.read_csv("../input/hpa-single-cell-image-classification/train.csv")
train_df

|   |   |
|---|---|
|ID | The base filename of the sample. As noted above all samples consist of four files - blue, green, red, and yellow.|
|Label | in the training data, this represents the labels assigned to each sample; in submission, this represent the labels assigned to each cell.|

|   |   |
|---|---|
|ID | サンプルのベースファイル名。上記のように、すべてのサンプルは、青、緑、赤、黄色の4つのファイルで構成されています。|
|Label | トレーニングデータでは、これは各サンプルに割り当てられたラベルを表します。提出時には、これは各セルに割り当てられたラベルを表します。|

## EDA

In [ ]:
path_to_train = '../input/hpa-single-cell-image-classification/train/'
train_df = pd.read_csv('../input/hpa-single-cell-image-classification/train.csv')
train_df.head()

In [ ]:
train_dataset_info = []
for name, labels in zip(train_df['ID'], train_df['Label'].str.split('|')):
    train_dataset_info.append({
        'path':os.path.join(path_to_train, name),
        'labels':np.array([int(label) for label in labels])})

In [ ]:
train_df_2 = pd.DataFrame(columns=["path", "Label"])
n = 0

for i, D in enumerate(train_dataset_info[:7000]):
    for x in D["labels"]:
        train_df_2.loc[n, "path"] = D["path"]
        train_df_2.loc[n, "Label"] = x
        n = n + 1 

train_df_2

In [ ]:
def ID_figure(LABEL_ID):
    pathlist_ID = []

    train_df_ID = train_df_2[train_df_2.Label == LABEL_ID]["path"].reset_index()

    for i, D in enumerate(train_df_ID["path"]):
        pathlist_ID.append(D)
        
    for i in range(12):
        plt.subplot(5, 3, i+1)
        
        red_image = cv2.imread(pathlist_ID[i]+"_red.png", cv2.IMREAD_UNCHANGED)
        green_image = cv2.imread(pathlist_ID[i]+"_green.png", cv2.IMREAD_UNCHANGED)
        blue_image = cv2.imread(pathlist_ID[i]+"_blue.png", cv2.IMREAD_UNCHANGED)
        #yellow_image = cv2.imread(pathlist_ID[i]+"_yellow.png", cv2.IMREAD_UNCHANGED)
        
        img_bgr = cv2.merge((blue_image, green_image, red_image))  
        
        plt.title(pathlist_ID[i][-44:])
        plt.imshow(img_bgr)
        plt.tick_params(labelbottom=False,
                        labelleft=False,
                        labelright=False,
                        labeltop=False)

In [ ]:
pathlist_ID = []
LABEL_ID = 0

train_df_ID = train_df_2[train_df_2.Label == LABEL_ID]["path"].reset_index()

for i, D in enumerate(train_df_ID["path"]):
    pathlist_ID.append(D)

### label == 0.Nucleoplasm 

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(0)

### 1.Nuclear membrane

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(1)

### 2.Nucleoli

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(2)

### 3.Nucleoli fibrillar center

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(3)

### 4.Nuclear speckles

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(4)

### 5.Nuclear bodies

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(5)

### 6.Nuclear bodies

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(6)

### 7.Endoplasmic reticulum

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(7)

### 8.Golgi apparatus

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(8)

### 9.Intermediate filaments

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(9)

### 10.Actin filaments 10. Microtubules

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(10)

### 11.Mitotic spindle

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(11)

### 12.Centrosome

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(12)

### 13.Plasma membrane

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(13)

### 14.Mitochondria

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(14)

### 15.Aggresome

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(15)

### 16.Cytosol

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(16)

### 17.Vesicles and punctate cytosolic patterns

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(17)

### 18.Negative

In [ ]:
fig = plt.figure(figsize=(20,24))
ID_figure(18)

## submission_test

In [ ]:
submission = pd.read_csv("../input/hpa-single-cell-image-classification/sample_submission.csv")
submission.to_csv('submission.csv', index=False)